In [4]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter.ttk import Progressbar
from asammdf import MDF
import pandas as pd
import tempfile
import threading

def filter_dbc(dbc_file):
    """Filters out lines starting with VAL_ from the DBC file."""
    with open(dbc_file, 'r') as file:
        lines = file.readlines()

    # Remove lines starting with VAL_
    filtered_lines = [line for line in lines if not line.strip().startswith("VAL_")]

    # Save the filtered DBC to a temporary file
    temp_dbc = tempfile.NamedTemporaryFile(delete=False, suffix=".dbc")
    with open(temp_dbc.name, 'w') as temp_file:
        temp_file.writelines(filtered_lines)

    return temp_dbc.name

def decode_and_save_mf4(mf4_file, dbc_file, new_mf4_file, root):
    # Filter the DBC file to exclude VAL_ lines
    filtered_dbc = filter_dbc(dbc_file)

    try:
        mdf = MDF(mf4_file)
        database_files = {"CAN": [(filtered_dbc, 0)], "LIN": []}
        decoded_mdf = mdf.extract_bus_logging(database_files=database_files)
        decoded_mdf.save(new_mf4_file)
        log_message(f"Decoded MF4 file saved as: {new_mf4_file}", root)
    finally:
        # Clean up the temporary DBC file
        os.unlink(filtered_dbc)

def process_mf4(mf4_file, dbc_file, output_folder, raster, root, progress_bar):
    file_name = os.path.basename(mf4_file)

    if not os.path.exists(mf4_file):
        log_message(f"Error: {mf4_file} does not exist.", root)
        return

    # Determine the relative path for the output file within the folder structure
    relative_path = os.path.relpath(mf4_file, start=input_path_entry.get())
    output_file_dir = os.path.join(output_folder, os.path.dirname(relative_path))

    if not os.path.exists(output_file_dir):
        os.makedirs(output_file_dir)

    output_file = os.path.join(output_file_dir, f"{file_name.replace('.mf4', '.csv')}")

    if os.path.exists(output_file):
        log_message(f"CSV file already exists, skipping: {output_file}", root)
        return

    try:
        decoded_mf4_file = os.path.join(output_file_dir, file_name)
        decode_and_save_mf4(mf4_file, dbc_file, decoded_mf4_file, root)

        mdf = MDF(decoded_mf4_file)  # Attempt to open the MF4 file
        resampled_mdf = mdf.resample(raster=raster)

        all_signals_df = pd.DataFrame()
        for signal in resampled_mdf:
            signal_name = signal.name
            signal_data = pd.DataFrame({
                'Timestamp': signal.timestamps,
                signal_name: signal.samples
            })
            if signal_name in all_signals_df.columns:
                signal_name = f"{signal_name}_dup"
            if all_signals_df.empty:
                all_signals_df = signal_data
            else:
                all_signals_df = pd.merge(all_signals_df, signal_data, on='Timestamp', how='outer')

        all_signals_df.to_csv(output_file, index=False)
        log_message(f"Resampled data from {mf4_file} successfully exported to {output_file}", root)

    except Exception as e:
        log_message(f"Error processing {mf4_file}: {e}", root)
        return

def process_folder(folder_path, dbc_file, output_folder, raster, root, progress_bar):
    mf4_files = []
    for root_dir, dirs, files in os.walk(folder_path):
        for dir_name in dirs:
            folder_output_path = os.path.join(output_folder, dir_name)
            if not os.path.exists(folder_output_path):
                os.makedirs(folder_output_path)

        mf4_files.extend([os.path.join(root_dir, file) for file in files if file.endswith('.mf4')])

    if not mf4_files:
        log_message("No MF4 files found.", root)
        return

    progress_bar['maximum'] = len(mf4_files)

    for idx, mf4_file in enumerate(mf4_files, start=1):
        process_mf4(mf4_file, dbc_file, output_folder, raster, root, progress_bar)
        progress_bar['value'] = idx
        root.after(10, root.update_idletasks)

def log_message(message, root):
    log_text.insert(tk.END, message + "\n")
    log_text.see(tk.END)
    root.after(10, root.update_idletasks)  # Explicitly update the root window after every log

def browse_file_or_folder():
    if file_or_folder_var.get() == "File":
        return filedialog.askopenfilename(filetypes=[("MF4 files", "*.mf4")])
    elif file_or_folder_var.get() == "Folder":
        return filedialog.askdirectory()

def browse_dbc_file():
    return filedialog.askopenfilename(filetypes=[("DBC files", "*.dbc")])

def browse_output_folder():
    return filedialog.askdirectory()

def start_processing():
    input_path = input_path_entry.get()
    dbc_file = dbc_file_entry.get()
    output_folder = output_folder_entry.get()
    raster = float(resampling_rate_entry.get())

    if not input_path or not dbc_file or not output_folder:
        messagebox.showerror("Error", "Please select all required inputs.")
        return

    log_message("Processing started...", root)

    # Start the processing in a separate thread to keep the GUI responsive
    processing_thread = threading.Thread(target=process_files, args=(input_path, dbc_file, output_folder, raster, root))
    processing_thread.start()

def process_files(input_path, dbc_file, output_folder, raster, root):
    if file_or_folder_var.get() == "File":
        process_mf4(input_path, dbc_file, output_folder, raster, root, progress_bar)
    elif file_or_folder_var.get() == "Folder":
        process_folder(input_path, dbc_file, output_folder, raster, root, progress_bar)

    messagebox.showinfo("Success", "Processing completed!")
    log_message("Processing completed!", root)

# Create the main window
root = tk.Tk()
root.title("MF4 File Processor")

# File or Folder selection
file_or_folder_var = tk.StringVar(value="File")
tk.Label(root, text="Process a single file or a folder?").grid(row=0, column=0, padx=10, pady=10)
tk.Radiobutton(root, text="File", variable=file_or_folder_var, value="File").grid(row=0, column=1)
tk.Radiobutton(root, text="Folder", variable=file_or_folder_var, value="Folder").grid(row=0, column=2)

# Input file/folder
tk.Label(root, text="Input file/folder:").grid(row=1, column=0, padx=10, pady=10)
input_path_entry = tk.Entry(root, width=50)
input_path_entry.grid(row=1, column=1, padx=10, pady=10)
tk.Button(root, text="Browse", command=lambda: input_path_entry.insert(0, browse_file_or_folder())).grid(row=1, column=2)

# DBC file
tk.Label(root, text="DBC file:").grid(row=2, column=0, padx=10, pady=10)
dbc_file_entry = tk.Entry(root, width=50)
dbc_file_entry.grid(row=2, column=1, padx=10, pady=10)
tk.Button(root, text="Browse", command=lambda: dbc_file_entry.insert(0, browse_dbc_file())).grid(row=2, column=2)

# Output folder
tk.Label(root, text="Output folder:").grid(row=3, column=0, padx=10, pady=10)
output_folder_entry = tk.Entry(root, width=50)
output_folder_entry.grid(row=3, column=1, padx=10, pady=10)
tk.Button(root, text="Browse", command=lambda: output_folder_entry.insert(0, browse_output_folder())).grid(row=3, column=2)

# Resampling rate
tk.Label(root, text="Resampling rate (seconds):").grid(row=4, column=0, padx=10, pady=10)
resampling_rate_entry = tk.Entry(root, width=10)
resampling_rate_entry.grid(row=4, column=1, sticky="w", padx=10, pady=10)
resampling_rate_entry.insert(0, "1.0")  # Default value

# Start button
tk.Button(root, text="Start Processing", command=start_processing).grid(row=5, column=1, pady=20)

# Progress bar
progress_bar = Progressbar(root, orient="horizontal", length=400, mode="determinate")
progress_bar.grid(row=6, column=0, columnspan=3, pady=10)

# Log window
log_text = tk.Text(root, height=10, width=80)
log_text.grid(row=7, column=0, columnspan=3, padx=10, pady=10)

# Start the GUI event loop
root.mainloop()
